In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:64"

In [ ]:
# 1) Odpoj vše, co by mohlo držet VRAM
import gc, torch
try:
    del model, optimizer, xb, yb
except:
    pass
gc.collect()
torch.cuda.empty_cache()


In [ ]:
# 2) Zkontroluj stav GPU
!nvidia-smi


Mon Oct 27 07:34:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P0             37W /   70W |     966MiB /  15360MiB |     90%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import tiktoken
import json

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1337)

Nastavení různých parametrů

In [ ]:
batch_size    = 8
block_size    = 256
n_embd        = 256
n_head        = 8
n_layer       = 4
max_iters     = 10000
eval_interval = 100
eval_iters    = 20
dropout       = 0.2
learning_rate = 3e-4

In [ ]:
# Downloading dataset
!wget -nc https://raw.githubusercontent.com/valkova-k/cactus-repo/refs/heads/main/assignment05/combined_books.txt

File ‘combined_books.txt’ already there; not retrieving.



Jelikož text obsahuje "Obsah", který se projevil při generování textu, může být žádoucí ho odstranit. Protože text obsahoval více příkladů dvoj-, troj- či dokonce čtyřteček, ale nejspíš za nimi nikdy nenásledovalo číslo, můžeme použít regulární výraz tvaru ".  .  .  {číslo}" a nechtěné odstranit. Totéž uděláme i s čísly stránek, které jsou tvaru "- {číslo} -".

In [ ]:
import re

pattern = r'(\.\s*\.\s*\.\s*\d+)|([–-]\s*\d+\s*[–-])'

with open('combined_books.txt', 'r', encoding='utf-8') as f:
    text = "".join(line for line in f if not re.search(pattern, line))


In [ ]:
print("length of dataset in characters: ", len(text))
print(text[:5000])

length of dataset in characters:  2304298
Produced by Miloslav Izar RUSKÁ KNIHOVNA IX. SPISY FEDORA MICHAJLOVIČE DOSTOJEVSKÉHO. Překlad rediguje JAROMÍR HRUBÝ Svazek I. ZÁPISKY Z MRTVÉHO DOMU. Přeložil H. JAROŠ. V PRAZE 1891. Tiskem a nákladem J. Otty. ČÁST PRVÁ. ÚVOD. V dalekých krajích Sibiře, uprostřed stepí, hor a neproniknutelných lesů vyskytují se zřídka malinká města s jedním nebo nanejvýš se dvěma tisíci obyvatelů, dřevěná to, neúhledná města se dvěma chrámy, jedním ve městě, druhým na hřbitově, a podobná více k slušné vesnici pod Moskvou než k městu. Bývají obyčejně hojně opatřena policejními hejtmany, komisary a ostatními podřízenými policejními dozorci. V Sibiři vůbec přes to, že je tam zima. jsou úřady neobyčejně teploučké. Lid tamní je prostý, nenačichlý liberálností; pořádky staré, pevné, staletími posvěcené. Úředníky, kteří právem hrají úlohu sibiřské šlechty, jsou buď tuzemci, zakořenělí Sibiřáci, anebo rodáci z evropského Ruska, zejména hlavních měst, kteří se dali při

Tokenizace rovnou přes tiktoken

In [ ]:
# tokenising
enc_name = "gpt2"
enc = tiktoken.get_encoding(enc_name)
encode = lambda s: enc.encode(s)
decode = lambda ids: enc.decode(ids)
vocab_size = enc.n_vocab

In [ ]:
data = torch.tensor(enc.encode(text), dtype=torch.long)

print(data.shape, data.dtype)

torch.Size([1342288]) torch.int64


Training/validation split




In [ ]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

## Blocks
= batching

In [ ]:
def get_batch(split, bs=None):
    src = train_data if split == 'train' else val_data
    bs = bs or batch_size
    ix = torch.randint(len(src) - block_size, (bs,))
    x = torch.stack([src[i:i+block_size] for i in ix])
    y = torch.stack([src[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device) # rovnou ukladam do device

## Loss estimate function


In [ ]:
@torch.no_grad()
def estimate_loss(model):
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = []
        for _ in range(eval_iters):
            X, Y = get_batch(split, bs=4)  # menší batch na evaluaci
            logits, loss = model(X, Y)
            losses.append(loss.item())     # jen číslo na CPU
        out[split] = sum(losses)/len(losses)
    model.train()
    return out

## Transformer

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)    # (B,T,head)
        q = self.query(x)  # (B,T,head)
        wei = q @ k.transpose(-2, -1) * (k.size(-1) ** -0.5)  # škálovaný dot-prod
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # causal mask
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)  # (B,T,head)
        out = wei @ v      # (B,T,T) @ (B,T,head) -> (B,T,head)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj  = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # (B,T,n_embd)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd_):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd_, 4 * n_embd_),
            nn.GELU(), # místo ReLU alternativní aktivační funkce GELU()
            nn.Linear(4 * n_embd_, n_embd_),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_embd_, n_head_):
        super().__init__()
        head_size = n_embd_ // n_head_
        self.sa   = MultiHeadAttention(n_head_, head_size)
        self.ffwd = FeedFoward(n_embd_)
        self.ln1  = nn.LayerNorm(n_embd_)
        self.ln2  = nn.LayerNorm(n_embd_)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTMini(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, n_layer, block_size, dropout):
        super().__init__()
        self.block_size = block_size
        self.token_embedding_table    = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[
            Block(n_embd, n_head) for _ in range(n_layer)
        ])
        self.ln_f   = nn.LayerNorm(n_embd)
        self.lm_head= nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)                         # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))  # (T,C)
        x = tok_emb + pos_emb                                             # (B,T,C)
        x = self.blocks(x)                                                # (B,T,C)
        x = self.ln_f(x)                                                  # (B,T,C)
        logits = self.lm_head(x)                                          # (B,T,V)
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(B*T))
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        # vždy crop na aktuální block_size instance
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / max(temperature, 1e-8)
            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = -float('inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

## Inicializace modelu

In [ ]:
vocab_size = enc.n_vocab

model = GPTMini(
    vocab_size=vocab_size,
    n_embd=n_embd,
    n_head=n_head,
    n_layer=n_layer,
    block_size=block_size,
    dropout=dropout
).to(device)

print(f"Model params: {sum(p.numel() for p in model.parameters())/1e6:.2f} M")


Model params: 29.00 M


## Uložení modelu

In [ ]:
SAVE_DIR  = "./final_model"   # default: lokálně v Colabu -> /content/final_model
MODEL_PATH= os.path.join(SAVE_DIR, "model_final.pt")
META_PATH = os.path.join(SAVE_DIR, "meta.json")
os.makedirs(SAVE_DIR, exist_ok=True)
print("Model will be saved to:", os.path.abspath(SAVE_DIR))

Model will be saved to: /content/final_model


## Optimalizátor + train

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

use_amp = (device == 'cuda')
scaler = torch.amp.GradScaler(enabled=use_amp)

model.train()  # pro jistotu, eval se přepíná v estimate_loss()

for it in range(max_iters):
    if it % eval_interval == 0 or it == max_iters - 1:
        losses = estimate_loss(model)  # ta si sama přepíná na eval a zpět
        print(f"step {it}: train {losses['train']:.4f}, val {losses['val']:.4f}")

    xb, yb = get_batch('train')
    optimizer.zero_grad(set_to_none=True)

    with torch.cuda.amp.autocast(enabled=use_amp):
        _, loss = model(xb, yb)   # <-- TADY MUSÍ BÝT ODSAZENÉ

    scaler.scale(loss).backward()
    # (volitelně) omezit gradienty, když by „utekly“
    # scaler.unscale_(optimizer)
    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    scaler.step(optimizer)
    scaler.update()


step 0: train 11.0098, val 11.0057


/tmp/ipython-input-3388894481.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


step 100: train 4.9034, val 4.8681
step 200: train 4.3141, val 4.3667
step 300: train 4.0301, val 4.0729
step 400: train 3.9035, val 3.9352
step 500: train 3.7990, val 3.8846
step 600: train 3.7245, val 3.7954
step 700: train 3.6148, val 3.7348
step 800: train 3.6551, val 3.6718
step 900: train 3.5600, val 3.6598
step 1000: train 3.5540, val 3.6475
step 1100: train 3.5260, val 3.6360
step 1200: train 3.4898, val 3.5649
step 1300: train 3.4627, val 3.5398
step 1400: train 3.4724, val 3.5562
step 1500: train 3.4245, val 3.5417
step 1600: train 3.4092, val 3.5245
step 1700: train 3.3834, val 3.4751
step 1800: train 3.4164, val 3.4841
step 1900: train 3.3844, val 3.4713
step 2000: train 3.3407, val 3.4703
step 2100: train 3.3430, val 3.4286
step 2200: train 3.3544, val 3.3575
step 2300: train 3.3131, val 3.3315
step 2400: train 3.2349, val 3.3946
step 2500: train 3.3117, val 3.3403
step 2600: train 3.2401, val 3.3400
step 2700: train 3.1813, val 3.2772
step 2800: train 3.1321, val 3.2733
s

## Generování z modelu

In [ ]:
seed = "\n"
context = torch.tensor([encode(seed)], dtype=torch.long, device=device)
out_ids = model.generate(context, max_new_tokens=500, temperature=0.85, top_k=50)[0].tolist()
print(decode(out_ids))


j�íct, tím v pověkně všechno, že
mohl přísní a konečný můžec… Mám, prosím, nebo može při, na
nemoc!“
„Posloupďte, Rodione Romanoviči,“ vydala se zárájevem a zašil se
nářstavovice.
„Byložáte, že čertu býtku, nďte to případ už vyhurtával… A tehdyž mám dává zatím… Co jsem
jsem taky proto, přece dalším. Mám že je odej, copak nyní se hloupé,
nevše je tím, že jsem vzala, tak šest máte za ním vám sobě na tím
náměříš… Co jsem jsem vám, převním jsem si nic nezáležila, že se ji zdát. Třepářila
jeho jak…“ pomyslil si a že ho nezdělával.
„Pořád tušení. A taky jsem se mě vám lídám, že jste přišla dal…“
„Ano? Co byste vy jenom těl? Což jeden zlíčený nájemnost,“ odpověděla. „Kterak
tam já o… Co víte? Co říkali je zlomížu! Dlouho! Hlavně jsem se
se n


## Uložení modelu

In [ ]:
def save_meta(path):
    meta = {
        "encoding_name": enc_name,   # <--- TADY ta změna
        "block_size": block_size,
        "n_embd": n_embd,
        "n_head": n_head,
        "n_layer": n_layer,
        "dropout": dropout,
    }
    with open(path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

save_meta(META_PATH)
torch.save(model.state_dict(), MODEL_PATH)
print(f"\nSaved model to: {MODEL_PATH}")
print(f"Saved meta to:   {META_PATH}")


Saved model to: ./final_model/model_final.pt
Saved meta to:   ./final_model/meta.json


In [ ]:
# stazeni na disk z colabu

from google.colab import files
files.download(MODEL_PATH)
files.download(META_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Použití uloženého modelu


In [ ]:
import os, json, torch, tiktoken

# --- Cesty k uloženému modelu ---
SAVE_DIR  = "./final_model"
MODEL_PATH = os.path.join(SAVE_DIR, "model_final.pt")
META_PATH  = os.path.join(SAVE_DIR, "meta.json")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# --- Načtení modelu + meta dat ---
if os.path.exists(MODEL_PATH) and os.path.exists(META_PATH):
    with open(META_PATH, "r", encoding="utf-8") as f:
        meta = json.load(f)

    # tokenizér podle meta
    enc = tiktoken.get_encoding(meta.get("encoding_name", "gpt2"))
    vocab_size = enc.n_vocab

    # architektura podle uložených parametrů
    block_size = meta["block_size"]
    n_embd     = meta["n_embd"]
    n_head     = meta["n_head"]
    n_layer    = meta["n_layer"]
    dropout    = meta["dropout"]

    # inicializace modelu se stejnými parametry
    model_loaded = GPTMini(
        vocab_size=vocab_size,
        n_embd=n_embd,
        n_head=n_head,
        n_layer=n_layer,
        block_size=block_size,
        dropout=dropout
    ).to(device)

    # načtení vah
    state = torch.load(MODEL_PATH, map_location=device)
    model_loaded.load_state_dict(state, strict=True)
    model_loaded.eval()

    # --- Generování textu ---
    seeds = [
    "\n",
    "Člověk je zvláštní tvor, pomyslel si Raskolnikov. ",
    "„Ty tomu nerozumíš,“ řekl tiše. ",
    "Ráno se probudil se zvláštním pocitem neklidu.",
    ]

    torch.manual_seed(42)  # pro reprodukovatelnost výběru tokenů

    for s in seeds:
      ctx = torch.tensor([enc.encode(s)], dtype=torch.long, device=device)
      out = model_loaded.generate(ctx, max_new_tokens=400, temperature=0.8, top_k=100)[0].tolist()
      print(f"\n=== SEED: {repr(s)} ===\n")
      print(enc.decode(out))


else:
    print("\n[INFO] Model/meta not found at:", MODEL_PATH, META_PATH)


=== SEED: '\n' ===


„Prosím,“ řekl náhle Razumi-
číze Porfirij, „a jak se jí předtím. Předtuchou, které mám
jsem se mohl odpravit.“
„Jakou zatím ještě daleko, jak zkormokračovat?“
Raskolnikov. „Ach vás budeš máš olouhým ústavkem a já s ním, že máte
se nyní, že říkám ženuje, podle vaší vám, o tom, které ho mi
zříkám jen proto, že může do smíchu,“ odpověděl. „Tak vás, proč, já jsem nenechám, že
dělám pročká pozorovat, že těsně odpovídám…“
„Copak…, že jste tam nejvíc s nimi nevědím…“
„A přece tak zítra, nebože mu jsem se tehdy posloučku. Mám
musím před ním vyšla, sběhla se mi, že mi jsi jí říct, ale neviděla jsem
je

=== SEED: 'Člověk je zvláštní tvor, pomyslel si Raskolnikov. ' ===

Člověk je zvláštní tvor, pomyslel si Raskolnikov.  Zatraceně  dokud
chvíli. „Tak, že jste vrah!“ začal prohlédl, „že tam?
že jsi děláš?“
„To je to všechno vůbec neptávám nikoho?“ vyhrklínal Razumichin.
„Ale kdy já nevyzpyt?“
„Jak to ochotě toho nedbát ostatně bytná, že známý… Ano!“
„Já není, protože vjete j

**Problematická místa - komentář**

Na začátku trénování GPT byl nejdřív trochu problém se zahlcením paměti GPU na Colabu, pomohlo ruční čištění a navolení nižších hodnot parametrů modelu, které jsme potom postupně zvyšovali. Hodně pro podobu vygenerovaných textů pomohly specifické seedy odpovídající např. typickým začátkům kapitol v románech.